## Layer Normalization

Ref: <a href='https://arxiv.org/abs/1607.06450'>Layer Normalization</a>

### Batch Normalization의 한계
배치 정규화는 미니 배치에 대해서 summed input의 평균과 분산을 계산하여 정규화를 수행한다. 이러한 방식은 내부 공변량 변화(Internal Covariate Shift)를 해결하여 일반화 개선 및 오버피팅 감소, 학습 안정성 향상 및 기울기 소실 문제 완화, 그리고 빠른 학습속도 및 가중치 초기화에 대해 강건해지는 장점을 가지지만 일부 단점이 존재한다.

1. 미니 배치 크기에 의존하며, 작은 크기의 미니 배치(ex - `batch_size = 1`)에 대해서는 원활히 동작하지 않게 된다. 최근의 LLM을 포함한 Large NLP Model들은 작은 크기의 배치 사이즈를 가지므로 이는 중요한 문제이다.
2. 각 시점(time step)마다 서로 다른 데이터가 연속적으로 입력되는 Sequential 형태의 데이터를 다루는 RNN 등 Recurrent 모델에서는 배치 정규화를 적용하기 어렵다.
3. 분산 학습 환경에서 기기간 평균 및 분산을 모두 계산하여야 하기 때문에 계산 효율성이 저하된다.
4. test 단계에서 inference 시 train 과정에서의 평균 및 분산을 계속 저장하여 대신 사용해주어야 한다.

### Layer Normalization
Layer Normalization은 BN의 한계점을 극복하기 위해 고안된 정규화 기법으로, 입력 데이터 전체 피처에 대해 평균 0, 분산 1을 갖도록 변환시킨다. 이는 BN에서 각 element마다 정규화되었던 것과 차이가 존재한다.

LN은 배치가 아닌 레이어를 기준으로 정규화를 수행함으로써 BN이 가지고 있던 배치 크기에 대한 의존도를 제거하였다. 또한 sequence에 따른 고정 길이 정규화가 이루어지기 때문에 Recurrent 기반 모델에도 적용이 수월하다. 
LN은 주로 NLP task에 많이 사용되며 트랜스포머 계열의 구현에서도 자주 사용된다.

입력 $X\in \mathbb{R}^{L\times B\times C}$에 대하여 $LN(X) = \gamma {X - \mathbb{E}_{C}[X] \over \sqrt{{Var}_{C}[X] + \epsilon}} + \beta$가 된다. (L, B의 값과 무관하게 계산)

In [6]:
from typing import Union, List

import torch
from torch import nn, Size

In [7]:
class LayerNorm(nn.Module):
    '''
    normalized_shape: element의 shape S에 대하여, 입력 X의 shape는 [*, S[0], S[1], ..., S[n]] (*는 차원 수 / sequence에서는 seq_len이 될 수 있음)
    eps: 엡실론
    elementwise_affine: 정규화된 값에 대해 스케일 및 시프트 적용 여부 결정
    '''
    def __init__(self, normalized_shape: Union[int, List[int], Size],
                 eps: float = 1e-5, elementwise_affine: bool = True):
        super().__init__()
        
        # Convert `normalized_shape` to `torch.Size`
        if isinstance(normalized_shape, int):
            normalized_shape = torch.Size([normalized_shape])
        elif isinstance(normalized_shape, list):
            normalized_shape = torch.Size(normalized_shape)
        assert isinstance(normalized_shape, torch.Size)
        
        self.normalized_shape = normalized_shape
        self.eps = eps
        self.elementwise_affine = elementwise_affine
        
        # gamma and beta for affine
        if self.elementwise_affine:
            self.gamma = nn.Parameter(torch.ones(normalized_shape))
            self.beta = nn.Parameter(torch.zeros(normalized_shape))
            
    def forward(self, x: torch.Tensor):
        assert self.normalized_shape == x.shape[-len(self.normalized_shape):]
        
        # dimensions to calculate mean and variance
        dims = [-(i + 1) for i in range(len(self.normalized_shape))]
        
        mean = x.mean(dim=dims, keepdim=True)
        mean_x2 = (x ** 2).mean(dim=dims, keepdim=True)
        
        var = mean_x2 - (mean ** 2)
        
        # layer normalize
        x_norm = (x-mean) / torch.sqrt(var + self.eps)
        
        # scale and shift
        if self.elementwise_affine:
            x_norm = self.gamma * x_norm + self.beta
        
        return x_norm

In [11]:
def _test():
    x = torch.randn([2,3,2,4]) # B C H W
    print(f'original input: {x})')
    
    
    ln = LayerNorm(x.shape[2:])
    x = ln(x)
    
    print(f'normalized input: {x}')
    print(f'{ln.gamma.shape = }')

In [12]:
if __name__ == '__main__':
    _test()

original input: tensor([[[[-1.0389, -0.5300, -0.2023,  0.7930],
          [ 1.1393,  0.2385,  0.8208, -2.2994]],

         [[-0.4791, -0.3841,  1.8926,  1.7519],
          [ 0.3365, -0.9453, -0.5782,  0.5030]],

         [[-0.1186, -0.1813, -0.4453,  0.3676],
          [ 0.8719, -1.2697,  0.1110, -0.0684]]],


        [[[-0.7527,  0.8848,  1.3261, -1.1935],
          [-1.3128, -2.1940,  0.6254,  0.3473]],

         [[-0.1483,  2.2627, -0.3401, -0.4508],
          [ 0.1664, -0.7996, -0.2658, -0.3954]],

         [[-0.2540, -0.6572, -0.4892, -1.1827],
          [ 1.4013,  0.0369,  0.3618,  0.8968]]]]))
normalized input: tensor([[[[-0.8430, -0.3684, -0.0629,  0.8653],
          [ 1.1881,  0.3482,  0.8911, -2.0184]],

         [[-0.7380, -0.6433,  1.6231,  1.4830],
          [ 0.0740, -1.2020, -0.8365,  0.2398]],

         [[-0.0465, -0.1543, -0.6085,  0.7901],
          [ 1.6577, -2.0269,  0.3485,  0.0399]]],


        [[[-0.4012,  0.9993,  1.3768, -0.7781],
          [-0.8801, -1.6338,  